In [25]:
from pykrx import stock  # pykrx 모듈에서 stock 모듈을 가져옵니다.
import pandas as pd  # pandas 모듈을 가져와서 pd라는 이름으로 사용합니다.
import FinanceDataReader as fdr  # FinanceDataReader 모듈을 가져와서 fdr이라는 이름으로 사용합니다.
from datetime import datetime  # datetime 모듈에서 datetime 클래스를 가져옵니다.
import os  # os 모듈을 가져옵니다.

# KRX(Korea Exchange)에서 상장된 모든 종목의 리스트를 가져옵니다.
stocks = fdr.StockListing('ETF/KR') # 한국 ETF 전종목 
# fdr.StockListing('KRX')


In [ ]:
stocks['Symbol'], type(stocks['Symbol'])
symbols_list = list(stocks['Symbol'])
symbols_list

# Test
df = fdr.DataReader('459580')
df




In [ ]:
# from pykrx import stock
# import pandas as pd
# import FinanceDataReader as fdr
# from datetime import datetime
# import os

# # 날짜 구간 설정
# start_date = '2023-01-01'  # 시작일
# end_date = '2024-01-09'    # 종료일 (현재 날짜 또는 원하는 날짜)

# # ETF 목록 가져오기
# stocks = fdr.StockListing('ETF/KR')
# symbols_list = list(stocks['Symbol'])

# # 결과를 저장할 빈 데이터프레임 생성
# all_etf_data = pd.DataFrame()

# for symbol in symbols_list:
#     try:
#         # 각 ETF의 기본 정보 가져오기
#         etf_info = stocks[stocks['Symbol'] == symbol].iloc[0]
        
#         # 지정된 기간의 ETF 가격 데이터 가져오기
#         df = fdr.DataReader(symbol, start=start_date, end=end_date)
        
#         # 기간 데이터 처리
#         period_data = {
#             '종목코드': symbol,
#             'ETF명': etf_info['Name'],
#             '시작일종가': df.iloc[0]['Close'],
#             '종료일종가': df.iloc[-1]['Close'],
#             '기간수익률(%)': ((df.iloc[-1]['Close'] - df.iloc[0]['Close']) / df.iloc[0]['Close'] * 100).round(2),
#             '기간최고가': df['High'].max(),
#             '기간최저가': df['Low'].min(),
#             '평균거래량': df['Volume'].mean().round(0),
#             # '시가총액': etf_info['Market'],
#             # '기초지수': etf_info['Underlying']
#         }
        
#         # 데이터프레임에 추가
#         all_etf_data = pd.concat([all_etf_data, pd.DataFrame([period_data])], ignore_index=True)
        
#         print(f"처리완료: {symbol} - {etf_info['Name']}")
        
#     except Exception as e:
#         print(f"에러 발생 ({symbol}): {str(e)}")

# # 수익률 기준으로 정렬
# all_etf_data = all_etf_data.sort_values('기간수익률(%)', ascending=False)

# # 결과 출력
# print(f"\n=== ETF 정보 ({start_date} ~ {end_date}) ===")
# print(all_etf_data)

# # CSV 파일로 저장
# filename = f'korea_etf_data_{start_date}_to_{end_date}.csv'
# all_etf_data.to_csv(filename, encoding='utf-8-sig', index=False)

# # 기본 통계 정보 출력
# print("\n=== 기간별 ETF 통계 ===")
# print(f"전체 ETF 수: {len(all_etf_data)}")
# print(f"평균 수익률: {all_etf_data['기간수익률(%)'].mean():.2f}%")
# print(f"최고 수익률 ETF: {all_etf_data.iloc[0]['ETF명']} ({all_etf_data.iloc[0]['기간수익률(%)']}%)")
# print(f"최저 수익률 ETF: {all_etf_data.iloc[-1]['ETF명']} ({all_etf_data.iloc[-1]['기간수익률(%)']}%)")


In [28]:
from pykrx import stock
import pandas as pd
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import numpy as np

# 날짜 구간 설정
start_date = '2020-01-01'
end_date = '2020-12-31'    # 예시 기간

# ETF 목록 가져오기
stocks = fdr.StockListing('ETF/KR')
symbols_list = list(stocks['Symbol'])

# 모든 ETF의 일별 수익률을 저장할 데이터프레임
daily_returns = pd.DataFrame()

# 각 ETF의 일별 수익률 계산
for symbol in symbols_list:
    try:
        # ETF 가격 데이터 가져오기
        df = fdr.DataReader(symbol, start=start_date, end=end_date)
        
        # 일별 수익률 계산 ((당일 종가 - 전일 종가) / 전일 종가)
        returns = df['Close'].pct_change()
        
        # ETF 이름 가져오기
        etf_name = stocks[stocks['Symbol'] == symbol].iloc[0]['Name']
        
        # 결과 데이터프레임에 추가
        daily_returns[etf_name] = returns
        
        print(f"처리완료: {symbol} - {etf_name}")
        
    except Exception as e:
        print(f"에러 발생 ({symbol}): {str(e)}")

# NA 값 제거
daily_returns = daily_returns.dropna()

# 상관관계 계산
correlation_matrix = daily_returns.corr()

# 결과 저장
# 1. 일별 수익률 저장
daily_returns.to_csv('daily_returns.csv', encoding='utf-8-sig')

# 2. 상관관계 매트릭스 저장
correlation_matrix.to_csv('correlation_matrix.csv', encoding='utf-8-sig')

# 상관관계 분석 결과
print("\n=== 상관관계 분석 결과 ===")

# 1. 가장 높은 양의 상관관계를 가진 ETF 쌍 찾기
high_corr = pd.DataFrame()
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        high_corr = pd.concat([high_corr, pd.DataFrame({
            'ETF1': [correlation_matrix.columns[i]],
            'ETF2': [correlation_matrix.columns[j]],
            'Correlation': [correlation_matrix.iloc[i,j]]
        })])

# 상관관계 절대값 기준으로 정렬
high_corr = high_corr.sort_values('Correlation', ascending=False)

print("\n상위 10개 높은 양의 상관관계:")
print(high_corr.head(10))

print("\n하위 10개 낮은 상관관계:")
print(high_corr.tail(10))

# 3. 기초 통계량 계산
print("\n=== 일별 수익률 기초 통계량 ===")
stats = pd.DataFrame({
    '평균 수익률(%)': daily_returns.mean() * 100,
    '표준편차(%)': daily_returns.std() * 100,
    '최대 수익률(%)': daily_returns.max() * 100,
    '최소 수익률(%)': daily_returns.min() * 100
})
print(stats.sort_values('평균 수익률(%)', ascending=False).head())

# 4. 히트맵 시각화 (선택적)
try:
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.figure(figsize=(20, 16))
    sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', center=0)
    plt.title('ETF 상관관계 히트맵')
    plt.tight_layout()
    plt.savefig('correlation_heatmap.png')
    plt.close()
    print("\n히트맵이 'correlation_heatmap.png'로 저장되었습니다.")
except:
    print("\n히트맵 생성을 위해서는 seaborn 라이브러리가 필요합니다.")



상위 10개 높은 양의 상관관계:
                ETF1                   ETF2  Correlation
0  KODEX CD금리액티브(합성)         TIGER 미국S&P500          NaN
0  KODEX CD금리액티브(합성)    TIGER CD금리투자KIS(합성)          NaN
0  KODEX CD금리액티브(합성)              KODEX 200          NaN
0  KODEX CD금리액티브(합성)          KODEX 머니마켓액티브          NaN
0  KODEX CD금리액티브(합성)         TIGER 미국나스닥100          NaN
0  KODEX CD금리액티브(합성)    KODEX KOFR금리액티브(합성)          NaN
0  KODEX CD금리액티브(합성)       KODEX 미국S&P500TR          NaN
0  KODEX CD금리액티브(합성)    TIGER KOFR금리액티브(합성)          NaN
0  KODEX CD금리액티브(합성)  TIGER 미국테크TOP10 INDXX          NaN
0  KODEX CD금리액티브(합성)   KODEX 종합채권(AA-이상)액티브          NaN

하위 10개 낮은 상관관계:
                    ETF1                   ETF2  Correlation
0    HANARO 글로벌백신치료제MSCI          TIGER 200 산업재          NaN
0    HANARO 글로벌백신치료제MSCI         TIGER BBIG레버리지          NaN
0    HANARO 글로벌백신치료제MSCI  SOL 한국형글로벌플랫폼&메타버스액티브          NaN
0    HANARO 글로벌백신치료제MSCI            RISE 창업투자회사          NaN
0          TIGER 200 산업재       

/usr/local/lib/python3.11/site-packages/seaborn/matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/usr/local/lib/python3.11/site-packages/seaborn/matrix.py:207: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)



히트맵이 'correlation_heatmap.png'로 저장되었습니다.
